## conda env : smplify_37

In [ ]:
!python smplifyx/main.py --config cfg_files/fit_smplx.yaml  \
    --data_folder ./data_folder  \
    --output_folder ./data_folder/output_folder  \
    --visualize="True" \
    --model_folder /media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/models \
    --vposer_ckpt /media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/vposer_v1_0 \
    --part_segm_fn /media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/smplx_parts_segm.pkl 

In [ ]:
# what is inside the /media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/smplx_parts_segm.pkl file
# it contains the part segmentation information for SMPL-X model
import pickle
with open('/media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/smplx_parts_segm.pkl', 'rb') as f:
    part_segm = pickle.load(f)
type(part_segm)

## To visualize the results

In [ ]:
! python smplifyx/render_results.py --mesh_fns ./data_folder/output_folder/meshes

# Understanding the outputs  ( using env: nlp_assg_env_latest)

it is giving two outputs. mesh ( .obj) and a pkl file. check if that pkl file is just the parameters of smlp-x

In [ ]:
import pickle

pkl_file = "/media/Ext_4T_SSD/ASHOK_PART2/smplify-x/data_folder/output_folder/results/lady_standing/000.pkl"
with open(pkl_file, 'rb') as f:
    data = pickle.load(f)
    # print(data.keys())
    
# print shapes of the data 
keys = data.keys()
for key in keys:
    print(key, data[key].shape, type(data[key]),sep=" :")
    

camera_rotation :(1, 3, 3) :<class 'numpy.ndarray'>
camera_translation :(1, 3) :<class 'numpy.ndarray'>
betas :(1, 10) :<class 'numpy.ndarray'>
global_orient :(1, 3) :<class 'numpy.ndarray'>
left_hand_pose :(1, 12) :<class 'numpy.ndarray'>
right_hand_pose :(1, 12) :<class 'numpy.ndarray'>
jaw_pose :(1, 3) :<class 'numpy.ndarray'>
leye_pose :(1, 3) :<class 'numpy.ndarray'>
reye_pose :(1, 3) :<class 'numpy.ndarray'>
expression :(1, 10) :<class 'numpy.ndarray'>
body_pose :(1, 32) :<class 'numpy.ndarray'>


In [ ]:
# print body_pose
print(data['body_pose'])

# add random noise to the body pose
import numpy as np
data['body_pose'] = data['body_pose'] + np.random.rand(*data['body_pose'].shape) * 0.2

# save the modified data into a new file
new_pkl_file = "/media/Ext_4T_SSD/ASHOK_PART2/smplify-x/data_folder/output_folder/results/lady_standing/000_modified.pkl"
with open(new_pkl_file, 'wb') as f:
    pickle.dump(data, f)

[[ 1.0882783  -0.25927648  0.07062989 -0.4454153  -0.6067004  -0.9026999
   0.81940883  0.30874324 -0.1470848   0.63804007 -1.8519192   0.3966687
  -0.49655244 -0.07236241  0.65701747 -1.424687   -1.0643927   0.66840994
  -0.2141063  -0.3877801   0.7207591  -0.25786936 -0.55952823 -0.12053697
   0.37059253  0.30171782  0.3720443   0.02662649  0.30333555  0.32818958
  -0.25504887  0.21502186]]


In [ ]:
# visualize the modified mesh using smpl viewer
!python smplifyx/render_results.py --mesh_fns /media/Ext_4T_SSD/ASHOK_PART2/smplify-x/data_folder/output_folder/results/lady_standing/000_modified.pkl

In [2]:
import pickle
modified_pkl = "/media/Ext_4T_SSD/ASHOK_PART2/smplify-x/data_folder/output_folder/results/lady_standing/000.pkl"
with open(modified_pkl, 'rb') as f:
    data = pickle.load(f)

data.keys()

#betas,expression,

dict_keys(['camera_rotation', 'camera_translation', 'betas', 'global_orient', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression', 'body_pose'])

In [4]:
data['betas'].shape

(1, 10)

In [6]:
import torch
import smplx

# Create the SMPL-X model
model_path = '/media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/models/smplx/SMPLX_NEUTRAL.npz'
model = smplx.create(model_path, model_type='smplx',
                     gender='neutral', ext='npz',
                     num_pca_comps=12,
                     create_global_orient=True,
                     create_body_pose=True,
                     create_betas=True,
                     create_left_hand_pose=True,
                     create_right_hand_pose=True,
                     create_expression=True,
                     create_jaw_pose=True,
                     create_leye_pose=True,
                     create_reye_pose=True,
                     create_transl=True)

# Define the batch size
batch_size = 1

# Define the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

# Generate random body parameters
betas = torch.randn(batch_size, 10, device=device)  # shape parameters
expression = torch.randn(batch_size, 10, device=device)  # expression parameters
# pose = torch.randn(batch_size, int(model.body_pose), device=device)  # pose parameters
pose = torch.randn(batch_size, 63, device=device)  # pose parameter

# Generate the SMPL-X mesh
# output = model(betas=betas, expression=expression, body_pose=pose[:, 3:], return_verts=True)
# output = model(betas=betas, expression=expression, body_pose=pose, return_verts=True)


# load from the data
betas = torch.tensor(data['betas'], device=device)
# body_pose = torch.tensor(data['body_pose'], device=device

import os.path as osp
from human_body_prior.tools.model_loader import load_vposer

use_vposer = True
dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vposer_ckpt = '/media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/vposer_v1_0'

if use_vposer:
    pose_embedding = torch.zeros([batch_size, 32],
                                    dtype=dtype, device=device,
                                    requires_grad=True)

    vposer_ckpt = osp.expandvars(vposer_ckpt)
    vposer, _ = load_vposer(vposer_ckpt, vp_model='snapshot')
    vposer = vposer.to(device=device)
    vposer.eval()


# body

output = model(betas=betas, expression=expression, return_verts=True, body_pose=body_pose)


import trimesh
import numpy as np

# Get the vertices and faces from the SMPL-X output
vertices = output.vertices.detach().cpu().numpy().squeeze()
faces = model.faces
out_mesh = trimesh.Trimesh(vertices, faces, process=False)
rot = trimesh.transformations.rotation_matrix(np.radians(180), [1, 0, 0])
out_mesh.apply_transform(rot)
out_mesh.show()



Found Trained Model: /media/Ext_4T_SSD/ASHOK_PART2/smplify-x/Downloads/vposer_v1_0/snapshots/TR00_E096.pt


RuntimeError: shape '[-1, 21, 3]' is invalid for input of size 32

In [31]:
# camera_rotation :(1, 3, 3) :<class 'numpy.ndarray'>
# camera_translation :(1, 3) :<class 'numpy.ndarray'>
# betas :(1, 10) :<class 'numpy.ndarray'>
# global_orient :(1, 3) :<class 'numpy.ndarray'>
# left_hand_pose :(1, 12) :<class 'numpy.ndarray'>
# right_hand_pose :(1, 12) :<class 'numpy.ndarray'>
# jaw_pose :(1, 3) :<class 'numpy.ndarray'>
# leye_pose :(1, 3) :<class 'numpy.ndarray'>
# reye_pose :(1, 3) :<class 'numpy.ndarray'>
# expression :(1, 10) :<class 'numpy.ndarray'>
# body_pose :(1, 32) :<class 'numpy.ndarray'>

# shape '[-1, 21, 3]' is invalid for input of size 65


32 +9+24

65